# 1. Introdução e Objetivo
A desigualdade social é uma das características mais marcantes do Brasil, especialmente quando observamos as diferenças entre as regiões do país. Estados do Norte e Nordeste apresentam, historicamente, indicadores socioeconômicos mais baixos quando comparados às regiões Sudeste e Centro-Oeste. Nesse cenário, o Índice de Desenvolvimento Humano (IDH) e a renda domiciliar per capita são indicadores importantes para compreender como as condições econômicas se relacionam com a qualidade de vida da população.

Este trabalho tem como objetivo investigar a relação entre o IDH e a renda per capita dos estados brasileiros, utilizando dados públicos referentes ao ano de 2010. A partir dessa análise, busca-se responder às seguintes perguntas:

**Perguntas de negócio:**
1. Existe correlação entre renda per capita e IDH dos estados brasileiros?
2. Estados com maior renda também apresentam maior IDH em seus componentes (educação, longevidade e renda)?
3. Em que medida a desigualdade regional pode ser explicada pela diferença de renda?

Os resultados esperados não são apenas numéricos, mas interpretativos: pretende-se discutir se renda, sozinha, explica o nível de desenvolvimento ou se outros fatores se tornam mais determinantes em determinados estados.
# 2. Coleta de Dados
Para a construção deste MVP foram utilizados dois conjuntos de dados públicos e oficiais, acessíveis gratuitamente. Ambos apresentam dados referentes ao ano de 2010, permitindo a comparação direta entre indicadores socioeconômicos.

### 2.1 Fonte — IDHM (Índice de Desenvolvimento Humano Municipal)
- **Origem:** Atlas do Desenvolvimento Humano no Brasil (PNUD, IPEA e Fundação João Pinheiro).
- **Link de acesso:** https://www.atlasbrasil.org.br/consulta/planilha
- **Descrição:** Tabela contendo o IDHM total e seus componentes (Educação, Longevidade e Renda) para todos os estados e municípios brasileiros.
- **Formato utilizado:** `.xlsx` exportado manualmente da plataforma e posteriormente convertido para `.csv` para uso no Databricks.
- **Ano considerado:** 2010.

### 2.2 Fonte — Renda média domiciliar per capita
- **Origem:** Plataforma governamental de estatísticas (IBGE / compilação por DATASUS).
- **Link de acesso:**- **Link de acesso:** http://tabnet.datasus.gov.br/cgi/tabcgi.exe?ibge/censo/cnv/rendauf
- **Descrição:** Valores de renda média domiciliar per capita por Unidade da Federação, com ajuste monetário pelo salário mínimo de 2010.
- **Formato utilizado:** `.csv`, criado a partir da extração manual fornecida pelo site.
- **Ano considerado:** 2010.

### 2.3 Considerações Éticas e de Uso
Os dados utilizados são:
- Publicamente acessíveis
- De uso livre para fins educacionais, científicos e acadêmicos
- Não contêm informações pessoais ou dados sensíveis

Por isso, não existe restrição ética ou legal para seu processamento, transformação e análise neste projeto.

# 3. Carga e Preparação dos Dados

Após a coleta manual, os arquivos foram ajustados e enviados para o Databricks Community, onde foram armazenados dentro do schema schema_v3. Durante esse processo, foram feitas correções manuais necessárias para preservar a consistência das tabelas, como a remoção de acentos e alteração de vírgula para ponto na separação das casas decimais.

### 3.1 Importação e visualização inicial das tabelas

Após o upload no Databricks, os arquivos originaram as seguintes tabelas:

schema_v3.idh_estados_atlas_v3

schema_v3.renda_estados_2010_csv

Exemplo de consulta inicial:

In [0]:
SELECT * FROM schema_v3.idh_estados_atlas_v3 LIMIT 5;



Estados,idhm_2010,idhm_renda_2010,idhm_longevidade_2010,idhm_educacao_2010
Acre,0.663,0.671,0.777,0.559
Alagoas,0.631,0.641,0.755,0.52
Amapa,0.708,0.694,0.813,0.629
Amazonas,0.674,0.677,0.805,0.561
Bahia,0.66,0.663,0.783,0.555


In [0]:
SELECT * FROM schema_v3.renda_estados_2010_csv LIMIT 5;

Estados,renda_media_2010
Rondonia,646.78
Acre,497.44
Amazonas,508.28
Roraima,578.38
Para,429.57


### 3.2 Limpeza e padronização dos dados

Como as correções importantes (acentos, duplicidades e padrão decimal) foram realizadas antes do upload, a criação das tabelas tratadas consistiu apenas em:

- Remover espaços excedentes
- Converter valores para tipo numérico adequado
- Criação das tabelas clean:


In [0]:
CREATE OR REPLACE TABLE schema_v3.idh_estados_clean_v3 AS
SELECT
    TRIM(Estados) AS Estados,
    CAST(idhm_2010 AS DOUBLE) AS idhm_2010,
    CAST(idhm_renda_2010 AS DOUBLE) AS idhm_renda_2010,
    CAST(idhm_longevidade_2010 AS DOUBLE) AS idhm_longevidade_2010,
    CAST(idhm_educacao_2010 AS DOUBLE) AS idhm_educacao_2010
FROM schema_v3.idh_estados_atlas_v3;


In [0]:
CREATE OR REPLACE TABLE schema_v3.renda_estados_clean_v3 AS
SELECT
    TRIM(Estados) AS Estados,
    CAST(renda_media_2010 AS DOUBLE) AS renda_media_2010
FROM schema_v3.renda_estados_2010_csv;


### 3.3 Verificação de consistência entre as bases

Para confirmar que os dados estavam corretos após o tratamento:

In [0]:
SELECT * FROM schema_v3.idh_estados_clean_v3 LIMIT 5;


Estados,idhm_2010,idhm_renda_2010,idhm_longevidade_2010,idhm_educacao_2010
Acre,0.663,0.671,0.777,0.559
Alagoas,0.631,0.641,0.755,0.52
Amapa,0.708,0.694,0.813,0.629
Amazonas,0.674,0.677,0.805,0.561
Bahia,0.66,0.663,0.783,0.555


In [0]:
SELECT * FROM schema_v3.renda_estados_clean_v3 LIMIT 5;

Estados,renda_media_2010
Rondonia,646.78
Acre,497.44
Amazonas,508.28
Roraima,578.38
Para,429.57


Como as bases já estavam consistentes, nenhuma inconsistência entre os estados foi encontrada, possibilitando a etapa seguinte de modelagem e integração.

### Modelagem dos Dados
Neste projeto, foi adotado um modelo em esquema estrela simples.

Foram criadas duas tabelas:
- **Dimensão**, com os estados do Brasil;
- **Fato**, com os indicadores de IDH e renda referentes ao ano de 2010.

A dimensão recebe uma chave artificial (`id_estado`), utilizada para relacionar cada estado aos seus indicadores na tabela fato.

#### 4.1 Criação da Dimensão de Estado

A dimensão contém uma linha por estado e um identificador numérico artificial (`id_estado`).  
Ela foi criada a partir da tabela tratada `idh_estados_clean_v3`.


In [0]:
-- 4.1 Criação da Dimensão de Estado
CREATE OR REPLACE TABLE schema_v3.dim_estado_v3 AS
WITH estados_distintos AS (
    SELECT DISTINCT Estados
    FROM schema_v3.idh_estados_clean_v3
)
SELECT
    ROW_NUMBER() OVER (ORDER BY Estados) AS id_estado,
    Estados
FROM estados_distintos;


Visualização opcional:

In [0]:
SELECT * FROM schema_v3.dim_estado_v3;


id_estado,Estados
1,Acre
2,Alagoas
3,Amapa
4,Amazonas
5,Bahia
6,Ceara
7,Distrito Federal
8,Espirito Santo
9,Goias
10,Maranhao


####4.2 Criação da Tabela Fato (IDH + Renda)
A tabela fato consolida os indicadores de IDH e renda por estado.
Ela é formada pela junção das tabelas tratadas com a dimensão criada.

In [0]:
-- 4.2 Criação da Tabela Fato: IDH + Renda
CREATE OR REPLACE TABLE schema_v3.fato_idh_renda_v3 AS
SELECT
    d.id_estado,
    i.idhm_2010,
    i.idhm_renda_2010,
    i.idhm_longevidade_2010,
    i.idhm_educacao_2010,
    r.renda_media_2010
FROM schema_v3.idh_estados_clean_v3 AS i
JOIN schema_v3.renda_estados_clean_v3 AS r
    ON i.Estados = r.Estados
JOIN schema_v3.dim_estado_v3 AS d
    ON i.Estados = d.Estados;


Visualização opcional:

In [0]:
SELECT * FROM schema_v3.fato_idh_renda_v3 LIMIT 10;


id_estado,idhm_2010,idhm_renda_2010,idhm_longevidade_2010,idhm_educacao_2010,renda_media_2010
1,0.663,0.671,0.777,0.559,497.44
2,0.631,0.641,0.755,0.52,421.32
3,0.708,0.694,0.813,0.629,575.42
4,0.674,0.677,0.805,0.561,508.28
5,0.66,0.663,0.783,0.555,481.18
6,0.682,0.651,0.793,0.615,445.88
7,0.824,0.863,0.873,0.742,1665.42
8,0.74,0.743,0.835,0.653,795.33
9,0.735,0.742,0.827,0.646,785.17
10,0.639,0.612,0.757,0.562,348.72


####4.3 Catálogo de Dados

######Tabela: schema_v3.dim_estado_v3

| Coluna    | Tipo   | Descrição                                                        |
| --------- | ------ | ---------------------------------------------------------------- |
| id_estado | INT    | Identificador artificial do estado (chave primária).             |
| Estados   | STRING | Nome do estado (sem acentos), conforme arquivo original tratado. |

######Tabela: schema_v3.fato_idh_renda_v3
| Coluna                | Tipo   | Descrição                                             |
| --------------------- | ------ | ----------------------------------------------------- |
| id_estado             | INT    | Chave estrangeira para `dim_estado_v3`.               |
| idhm_2010             | DOUBLE | IDH total do estado.                                  |
| idhm_renda_2010       | DOUBLE | Componente de renda do IDH.                           |
| idhm_longevidade_2010 | DOUBLE | Componente de longevidade do IDH.                     |
| idhm_educacao_2010    | DOUBLE | Componente de educação do IDH.                        |
| renda_media_2010      | DOUBLE | Renda média domiciliar per capita em 2010 (em reais). |




### 5 Análise  de dados
A análise é dividida em duas etapas:
- Qualidade dos Dados
- Respostas às perguntas definidas no objetivo

O foco é ser objetivo, usando SQL para responder às questões e interpretações curtas após cada bloco.

#### 5.1 Qualidade dos Dados
Nesta etapa, verificamos se há valores nulos, duplicidades ou inconsistências que possam prejudicar a análise.

In [0]:
-- Contagem de nulos por coluna na tabela fato
SELECT
  SUM(CASE WHEN id_estado IS NULL THEN 1 END) AS nulos_id_estado,
  SUM(CASE WHEN idhm_2010 IS NULL THEN 1 END) AS nulos_idhm,
  SUM(CASE WHEN idhm_renda_2010 IS NULL THEN 1 END) AS nulos_idhm_renda,
  SUM(CASE WHEN idhm_longevidade_2010 IS NULL THEN 1 END) AS nulos_idhm_longevidade,
  SUM(CASE WHEN idhm_educacao_2010 IS NULL THEN 1 END) AS nulos_idhm_educacao,
  SUM(CASE WHEN renda_media_2010 IS NULL THEN 1 END) AS nulos_renda
FROM schema_v3.fato_idh_renda_v3;


nulos_id_estado,nulos_idhm,nulos_idhm_renda,nulos_idhm_longevidade,nulos_idhm_educacao,nulos_renda
null,null,null,null,null,null


Interpretação:
Se todos os valores retornarem NULL = 0, a base está completa e pronta para análise.

####5.1.2 Verificar duplicidade de estados

In [0]:
SELECT id_estado, COUNT(*) AS registros
FROM schema_v3.fato_idh_renda_v3
GROUP BY id_estado
HAVING COUNT(*) > 1;


id_estado,registros


Interpretação:
Nenhum resultado significa que cada estado aparece exatamente uma vez.

####5.1.3 Verificar intervalos dos indicadores

Nesta etapa, avaliamos se os valores mínimos e máximos dos indicadores numéricos estão dentro de uma faixa esperada, de acordo com os dados oficiais do Brasil em 2010. Isso ajuda a confirmar que não houve erro de conversão, digitação ou carregamento dos dados.
Consulta utilizada:

In [0]:
SELECT
  MIN(idhm_2010) AS min_idh,
  MAX(idhm_2010) AS max_idh,
  MIN(renda_media_2010) AS min_renda,
  MAX(renda_media_2010) AS max_renda
FROM schema_v3.fato_idh_renda_v3;


min_idh,max_idh,min_renda,max_renda
0.631,0.824,348.72,1665.42



###Interpretação:


- O IDH varia entre 0.631 e 0.824, o que está dentro do intervalo real observado entre os estados brasileiros em 2010.
- A renda per capita varia entre R$ 348,72 e R$ 1.665,42, valor também coerente com os extremos nacionais da época.


###Conclusão:
Os valores mínimos e máximos estão dentro da realidade oficial, o que indica que os dados foram carregados corretamente e não há distorções numéricas. A base está pronta para as análises seguintes.

###5.2 Solução do Problema e Respostas às Perguntas
A seguir, cada pergunta definida no objetivo é respondida com consultas SQL e uma interpretação objetiva.

####5.2.1 Existe correlação entre renda per capita e IDH dos estados brasileiros?



In [0]:
SELECT CORR(renda_media_2010, idhm_2010) AS correlacao_idh_renda
FROM schema_v3.fato_idh_renda_v3;

correlacao_idh_renda
0.9410295844618911


###Interpretação:
O coeficiente de correlação varia de -1 a 1.
Valores positivos próximos de 1 indicam relação direta forte entre renda e IDH.
O valor obtido indica correlação positiva, mostrando que estados mais ricos tendem a ter maior IDH.

###5.2.2 Estados com maior renda também apresentam maior IDH?
Ranking por renda:


In [0]:
SELECT
  e.Estados,
  f.renda_media_2010,
  f.idhm_2010
FROM schema_v3.fato_idh_renda_v3 f
JOIN schema_v3.dim_estado_v3 e
  ON f.id_estado = e.id_estado
ORDER BY f.renda_media_2010 DESC;


Estados,renda_media_2010,idhm_2010
Distrito Federal,1665.42,0.824
Sao Paulo,1036.51,0.783
Rio de Janeiro,993.21,0.761
Santa Catarina,967.45,0.774
Rio Grande do Sul,940.28,0.746
Parana,870.59,0.749
Espirito Santo,795.33,0.74
Goias,785.17,0.735
Mato Grosso do Sul,784.97,0.729
Mato Grosso,735.32,0.725


###Ranking por IDH total:

In [0]:
SELECT
  e.Estados,
  f.idhm_2010,
  f.renda_media_2010
FROM schema_v3.fato_idh_renda_v3 f
JOIN schema_v3.dim_estado_v3 e
  ON f.id_estado = e.id_estado
ORDER BY f.idhm_2010 DESC;


Estados,idhm_2010,renda_media_2010
Distrito Federal,0.824,1665.42
Sao Paulo,0.783,1036.51
Santa Catarina,0.774,967.45
Rio de Janeiro,0.761,993.21
Parana,0.749,870.59
Rio Grande do Sul,0.746,940.28
Espirito Santo,0.74,795.33
Goias,0.735,785.17
Minas Gerais,0.731,733.24
Mato Grosso do Sul,0.729,784.97


###Interpretação
A comparação dos rankings mostra que estados com maior renda, como Distrito Federal e São Paulo, também aparecem entre os maiores valores de IDH. No entanto, observa-se que Santa Catarina apresenta IDH total superior ao do Rio de Janeiro, mesmo possuindo renda média inferior. Isso indica que, embora exista uma relação positiva entre renda e desenvolvimento humano, a renda por si só não explica completamente o nível de IDH, e outros fatores podem influenciar.

###5.2.3 Qual componente do IDH mais se relaciona com a renda?
Nesta etapa, avaliamos qual dos componentes do IDH apresenta maior correlação com a renda média domiciliar per capita.
Consulta utilizada:

In [0]:
SELECT
    CORR(renda_media_2010, idhm_renda_2010) AS corr_idh_renda,
    CORR(renda_media_2010, idhm_educacao_2010) AS corr_idh_educacao,
    CORR(renda_media_2010, idhm_longevidade_2010) AS corr_idh_longevidade
FROM schema_v3.fato_idh_renda_v3;


corr_idh_renda,corr_idh_educacao,corr_idh_longevidade
0.9710672526759295,0.8829372909121291,0.8918164191016149


###Análise:
Os resultados mostram que a renda média domiciliar per capita apresenta correlação muito forte com o componente IDH Renda (≈ 0,97), o que era esperado, já que ambos refletem diretamente as condições econômicas dos estados.
Observa-se também uma correlação alta com os componentes de Educação (≈ 0,88) e Longevidade (≈ 0,89). Isso indica que estados mais ricos tendem, em geral, a apresentar melhores indicadores educacionais e maior expectativa de vida.
No entanto, a correlação com educação e longevidade é inferior à observada com o componente de renda, sugerindo que, embora a renda influencie essas dimensões, outros fatores sociais e estruturais também desempenham papel relevante. Assim, a renda explica fortemente o componente econômico do IDH, mas não é o único determinante do desenvolvimento humano.


###5.2.4 Estados com baixa renda apresentam bom desempenho em educação ou longevidade?
Nesta etapa, analisamos se existem estados com renda média baixa que, ainda assim, apresentam valores relativamente elevados nos componentes de Educação ou Longevidade do IDH. Isso ajuda a avaliar se é possível obter bons resultados sociais mesmo em contextos econômicos menos favoráveis.
Consulta utilizada (ordenando pelos menores valores de renda):

In [0]:
SELECT
  e.Estados,
  f.renda_media_2010,
  f.idhm_educacao_2010,
  f.idhm_longevidade_2010
FROM schema_v3.fato_idh_renda_v3 f
JOIN schema_v3.dim_estado_v3 e
  ON f.id_estado = e.id_estado
ORDER BY f.renda_media_2010 ASC;


Estados,renda_media_2010,idhm_educacao_2010,idhm_longevidade_2010
Maranhao,348.72,0.562,0.757
Piaui,408.27,0.547,0.777
Alagoas,421.32,0.52,0.755
Para,429.57,0.528,0.789
Ceara,445.88,0.615,0.793
Paraiba,462.29,0.555,0.783
Bahia,481.18,0.555,0.783
Acre,497.44,0.559,0.777
Sergipe,508.2,0.56,0.781
Amazonas,508.28,0.561,0.805


###Análise 
A ordenação dos estados por renda média per capita evidencia que Maranhão, Piauí, Alagoas e Pará estão entre os menores valores do país. No entanto, ao observar os componentes do IDH, nota-se que alguns desses estados apresentam indicadores de longevidade relativamente elevados, como o Ceará (≈ 0,793) e o Pará (≈ 0,789), mesmo com renda inferior a R$ 450.
Isso indica que, embora a renda seja um fator importante, ela não é o único determinante do desenvolvimento humano. Resultados em saúde e expectativa de vida podem ser influenciados por políticas públicas e condições sociais que não dependem exclusivamente do nível de renda.

###5.2.5 Identificação de estados destoantes
Nesta etapa, buscamos identificar estados que se desviam do padrão geral observado entre renda e desenvolvimento humano, ou seja, casos em que os componentes do IDH apresentam comportamento diferente do que seria esperado dado o nível de renda.
A ideia é comparar o componente de renda do IDH com os componentes de educação e longevidade.

In [0]:
SELECT
  e.Estados,
  f.renda_media_2010,
  f.idhm_2010,
  ABS(f.idhm_renda_2010 - f.idhm_educacao_2010) AS dif_renda_educacao,
  ABS(f.idhm_renda_2010 - f.idhm_longevidade_2010) AS dif_renda_longevidade
FROM schema_v3.fato_idh_renda_v3 f
JOIN schema_v3.dim_estado_v3 e
  ON f.id_estado = e.id_estado
ORDER BY dif_renda_educacao DESC;


Estados,renda_media_2010,idhm_2010,dif_renda_educacao,dif_renda_longevidade
Rondonia,646.78,0.69,0.135,0.08800000000000008
Rio Grande do Sul,940.28,0.746,0.127,0.07099999999999995
Alagoas,421.32,0.631,0.121,0.11399999999999999
Distrito Federal,1665.42,0.824,0.121,0.010000000000000009
Para,429.57,0.646,0.118,0.14300000000000002
Amazonas,508.28,0.674,0.11599999999999999,0.128
Acre,497.44,0.663,0.11199999999999999,0.10599999999999998
Sergipe,508.2,0.665,0.11199999999999999,0.10899999999999999
Mato Grosso do Sul,784.97,0.729,0.11099999999999999,0.09299999999999997
Bahia,481.18,0.66,0.10799999999999998,0.12


###Análise:
A ordenação dos estados pela diferença entre o componente de renda do IDH e o componente de educação evidencia alguns casos de maior desvio do padrão geral. Estados como Rondônia (≈ 0,135), Rio Grande do Sul (≈ 0,127) e Alagoas (≈ 0,121) apresentam diferenças elevadas entre os dois componentes, indicando que o desempenho educacional não acompanha, na mesma proporção, o nível de renda medido pelo IDH.
Observa-se também que estados como Pará e Amazonas apresentam diferenças relevantes entre o componente de renda e o de longevidade, sugerindo que, apesar de níveis econômicos mais baixos, esses estados conseguem resultados relativamente melhores em expectativa de vida.
Esses casos reforçam que, embora a renda tenha forte relação com o desenvolvimento humano, existem exceções importantes, e que o IDH deve ser analisado de forma multidimensional, considerando separadamente educação, longevidade e renda.

##6. Conclusão e Autoavaliação
###6.1 Conclusão
O objetivo deste MVP foi analisar a relação entre a renda média domiciliar per capita e o nível de desenvolvimento humano dos estados brasileiros em 2010, utilizando o IDH e seus componentes como indicadores.
A partir da modelagem e das análises realizadas, foi possível observar que:
- Existe uma correlação positiva forte entre renda e IDH total, indicando que estados mais ricos tendem a apresentar maior nível de desenvolvimento humano.
- O componente de renda do IDH é o que apresenta maior correlação com a renda média, como esperado.
- Os componentes de educação e longevidade também apresentam correlação positiva, porém menos intensa, mostrando que o desenvolvimento humano é influenciado por outros fatores além da renda.
- Foram identificados casos em que estados com renda mais baixa apresentam desempenho relativamente melhor em longevidade, assim como estados em que a educação não acompanha o nível econômico, reforçando o caráter multidimensional do IDH.

Assim, os resultados indicam que a renda é um fator importante para explicar o desenvolvimento humano, mas não é suficiente, por si só, para explicar todas as diferenças observadas entre os estados brasileiros.

###6.2 Autoavaliação
Este trabalho representou um primeiro contato mais aprofundado com a construção de um pipeline de dados em ambiente de nuvem. Até então, minha experiência com dados estava concentrada principalmente na utilização de SQL para consultas sobre bases já estruturadas, sem envolvimento direto com etapas de coleta, carga, modelagem e organização dos dados.
Por esse motivo, optei por trabalhar com bases públicas e relativamente simples, com poucos atributos e granularidade por estado, o que permitiu focar no entendimento do processo completo de um MVP de dados: desde a obtenção dos arquivos, passando pela ingestão no Databricks, tratamento, criação de tabelas limpas, modelagem em esquema estrela e, por fim, a análise dos resultados.

Os principais desafios encontrados foram:
- Padronização dos dados durante a importação dos CSVs (acentos, separadores decimais e duplicidades);
- Definição de uma modelagem simples, mas coerente com os conceitos de Data Warehouse;
- Interpretação dos resultados de forma crítica, indo além dos números.

Apesar dessas dificuldades, considero que os objetivos definidos no início do trabalho foram atingidos, pois foi possível responder às perguntas propostas e compreender melhor a relação entre renda e desenvolvimento humano nos estados brasileiros.
Como trabalhos futuros, seria interessante:
- Incluir dados de outros anos para análise temporal;
- Aumentar o nível de granularidade para municípios;
- Integrar novos indicadores socioeconômicos, como saúde, desigualdade de renda ou investimento público;
- Explorar visualizações para complementar a análise.

De forma geral, este MVP contribuiu significativamente para o aprendizado prático sobre pipelines de dados e para o desenvolvimento das habilidades necessárias para projetos mais complexos na área de dados.